# Lecture 04 : A Few More Things

written by SinsuSquid (bgkang) on 27 November 2024

아마 오늘이 마지막 시간이 되지 않을까 싶은데, 오늘은 PyTorch와 PyG를 사용하기 위해 알아두면 좋고 너무 필요하지만 main 주제와는 조금 거리가 있어서 지금까지 말하지 않고 넘어간 부분들에 대한 이야기를 해볼까 합니다. 앞에 부분에 비하면야 거의 껌에 가까운 내용들이 될 것 같으니, 편하게 의자 제끼고 들어보면 될 것 같아요. (진짜 의자 제끼고 들으면 화낼거에요!)

## On GPUs (maybe CUDA)

저번 시간에 왜 인공지능 model training 과정에서 GPU가 필요한지에 대한 이야기를 해보았던 것 같아요. 그래서 PyTorch에서 GPU를 사용하는 방법들에 대해 알아보도록 하겠습니다. 연구실에 NVIDIA GPU가 많다 보니 주로 CUDA에 대한 이야기를 할 것 같은데, Apple MPS의 경우에도 (제발) 비슷한 형태로 code가 구현되어 있기 떄문에 적당히 수정해서 적용해보면 될 것 같아요. 참고로, 이 tutorial에서 benchmarking까지는 하지 않을 생각이니까, 실제로 얼마나 빨라지는지는 당신이 직접 확인해보길 바랄게요.

In [ ]:
# cuda or mps availability부터 확인해야겠죠?
import torch
print(torch.cuda.is_available()) # 내 컴퓨터가 NVIDIA GPU를 사용할 때
print(torch.backends.mps.is_available()) # 내 컴퓨터가 APPLE MPS GPU를 사용할 때

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:1')
    # 만약 당신이 GPU여러개를 쓰는 부자라면,
    # device = torch.device('cuda:{GPU_ID}")
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

그렇다면 실제로 어떻게 GPU를 사용하는지 한번 확인해볼게요. GPU 사용을 위해서는 모든 `torch.tensor`가 GPU-ready 상태인지 확인해야 합니다. 예시를 통해 살펴볼게요.

In [ ]:
cpu_tensor = torch.tensor([0.0])
gpu_tensor = torch.tensor([0.0]).to(device) # method 하나로 간단하게 GPU-ready!
# 사랑해요 PyTorch!

print(cpu_tensor)
print(gpu_tensor) # attribute중에 'device'라는게 새로 생겼죠?

다음엔 model을 GPU-ready 상태로 만들어볼게요. Model이 GPU-ready하다는 말은 model 안에 있는 모든 layer는 input과 output을 GPU-ready한 형태로 받아들이고 생성한다는 뜻입니다.

In [ ]:
class SimpleModel(torch.nn.Module):
    def __init__(self, in_size, out_size):
        super().__init__()
        self.in_size = in_size
        self.out_size = out_size

        self.linear = torch.nn.Linear(self.in_size, self.out_size)
        self.relu = torch.nn.ReLU()

        self.reset_parameters()

        return

    def forward(self, x):
        x = self.linear(x)
        return self.relu(x)

    def reset_parameters(self):
        self.linear.reset_parameters()

In [ ]:
cpu_model = SimpleModel(10, 1)
gpu_model = SimpleModel(10, 1).to(device)

print(next(cpu_model.parameters()))
print(next(gpu_model.parameters())) # 'device' 확인

GPU를 사용하는 경우 신경써줘야 할 한가지가 있는데, 바로 `torch.tensor`를 `numpy.ndarray`로 바꿔주어야 하는 경우에요. 물론 version update가 되면서 여러분이 이걸 보고 있을 떄는 별 문제가 없을수도 있지만, `torch.tensor` (on CUDA) -> `torch.tensor` (on CPU) -> `numpy.ndarray`의 단계를 거쳐야 `TypeError`가 발생하지 않습니다.

In [ ]:
gpu_tensor = torch.tensor([0.0, 1.0, 2.0, 3.0]).to(device)
gpu_tensor += gpu_tensor # 뭐라도 gradient를 만들어보려고...
# gpu_tensor.numpy() - This doesn't work!
gpu_tensor.to('cpu').numpy() # Yay!

## Early Stopping

여기서 early stopping이란 technique이 뭔지 설명하진 않을거에요. 물론 early stopping을 쓰면 안된다! 라고 주장하시는 분들도 계시지만 그래도 일단 개발 단계에서는 편리하니까요. 왜 그런지는 잘 모르겠지만 별도의 기능으로 구현되어 있진 않고 user-define해서 사용해야 합니다. 사실 구현이 어려운건 아니니까요, 간단하게 training_loss가 10 epoch동안 개선되지 않는 경우 EarlyStop하게끔 `run` 함수를 구성해봅시다.

In [ ]:
trn_loss = []; val_loss = []

def run(epochs):
    for e in range(epochs):
        trn_loss.append(train(model, train_loader))
        val_loss.append(validation(model, validation_loader))

        print(f"Epoch : {e:05d} | Trn. Loss : {trn_loss[-1]:.3f} | Val. Loss : {val_loss[-1]:.3f}")
        
        if (epoch > 10) and (trn_loss[epoch - 10] < trn_loss[epoch]): # 10 epoch 사이에 개선이 없으면
            print("Reached EarlyStopping Limit!")
            break # 참 쉽죠?

    print("Training Complete! >:D")

## Learning Rate Scheduler

효과적인 학습을 위해서는 학습 초기에는 learning rate를 크게 했다가 epoch가 진행되면서 감소시키는게 바람직하겠죠. 이런 역할을 해주는게 바로 learning rate scheduler입니다. 여러 algorithm들이 있지만, 여기서는 `ReduceLROnPlateau` (직관적이죠? Plateau에 도달하면 LR을 줄이겠다.)을 사용해볼게요. 다행히도 이거까지 내가 code로 짤 필요는 없어요. 사용법은 [ReduceLROnPlateau](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html)에서 확인해보세요.

In [ ]:
model = SimpleModel(10, 1)
optimizer = torch.optim.Adam(model.parameters(), lr = 1E-2)
criterion = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience = 10)
# 여러 option들이 있지만, factor랑 patience정도만 알아두면 충분할거에요.

In [ ]:
def run(epochs):
    for e in range(epochs):
        trn_loss.append(train(model, train_loader))
        val_loss.append(validation(model, validation_loader))

        scheduler.step(val_loss[epoch]) # code 한줄로 간단하게!

        print(f"Epoch : {e:05d} | Trn. Loss : {trn_loss[-1]:.3f} | Val. Loss : {val_loss[-1]:.3f}")
        

    print("Training Complete! >:D")

## Outtro

자, 여기까지 해서 제가 하고싶었던 이야기들은 마무리가 됩니다. 한가지 추천을 좀 해보자면, 이제 code를 이해할 수 있는 단계가 되었으니 [Chemomile](https://github.com/SinsuSquid/Chemomile) (네, 홍보 맞습니다.)에 들어가서 `src` 안에 있는 파일들을 한번 읽어보는 걸 추천해볼게요. 제가 지금까지 적어놓은 내용들이 아마 많은 부분 Chemomile에도 구현이 되어있을거에요. 다시 말해, 여러분은 이제 제가 알고있던 모든 지식을 똑같이 전수받은 셈입니다. 저는 이제 여러분들이 배운 내용들이 나중에 어떤 방향에서 사용되고 있을지를 두근두근한 마음으로 기다리고 있을게요. 들꽃반 친구들 지금까지 고생 많았어요! 😍